**Importing packages that we need**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, njit, vectorize

**Setting up the Monte Carlo method for a NxN lattice:**

In [11]:
#function that defines that inital state for the lattice calling a random number generator
@jit
def StateInitalize(N):
    #NXN number of spins
    Lattice = np.zeros((N,N))
    for i in range(N):
        for l in range(N):
            randNum = np.random.random()
            if randNum < 0.5:
                Lattice[i,l] = int(-1)
            else:
                Lattice[i,l] = int(1)
                
    return Lattice

In [29]:
A = StateInitalize(4)
print(A)

[[ 1. -1. -1. -1.]
 [ 1. -1. -1.  1.]
 [ 1. -1. -1. -1.]
 [-1. -1.  1. -1.]]


**NeigbourInteraction is a function that calculates the change in energy upon flipping one spin:**

In [43]:
def NeigbourInteraction(line,column,InitialState,LatticeSize):
    M = LatticeSize
    DeltaE = 0
    if line == int(0):#if at the top edge of lattice
        #State to the left
        if column == int(0):
            Temp1 = InitialState[line,column]*InitialState[line,column+1]#to the left
        else:
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
        #State to the right
        if column == int((M-1)):
            Temp2 = InitialState[line,column]*InitialState[line,column-1]#to the right
        else:
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
        #State above
        Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
        #State below
        Temp4 = InitialState[line,column]*InitialState[line+1,column]#below
    if line == int((M-1)):#if at the bottom edge lattice
        #State to the left
        if column == int(0):
            Temp1 = InitialState[line,column]*InitialState[line,column+1]#to the left
        else:
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
        #State to the right
        if column == int((M-1)):
            Temp2 = InitialState[line,column]*InitialState[line,column-1]#to the right
        else:
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
        #State above
        Temp3 = InitialState[line,column]*InitialState[line-1,column]#above
        #State below
        Temp4 = InitialState[line,column]*InitialState[line-1,column]#below
    #Then check if at left or right edges or inside
    else:
        #State to the left
        if column == int(0):
            Temp1 = InitialState[line,column]*InitialState[line,column+1]#to the left
        else:
            Temp1 = InitialState[line,column]*InitialState[line,column-1]#to the left
        #State to the right
        if column == int((M-1)):
            Temp2 = InitialState[line,column]*InitialState[line,column-1]#to the right
        else:
            Temp2 = InitialState[line,column]*InitialState[line,column+1]#to the right
        #State above
        Temp3 = InitialState[line,column]*InitialState[line+1,column]#above
        #State below
        Temp4 = InitialState[line,column]*InitialState[line-1,column]#below

    DeltaE = -(Temp1 + Temp2 + Temp3 + Temp4)

    return DeltaE

**Calculating the energy of the lattice according to the Ising model with periodic boundary conditions:**

In [49]:
def LatticeEnergyIsing(InitialState,LatticeSize):
    M = LatticeSize
    Energy = 0
    for i in range(M-1):#loop over lines
        for j in range(M-1):#loop over columns
            if (i == 0 and j == 0):#if at the top and left edge of lattice
                Temp1 = InitialState[i,j]*InitialState[i,j+1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j+1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i+1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if (i == 0 and j == 0):#if at the top and right edge of lattice
                Temp1 = InitialState[i,j]*InitialState[i,j-1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j-1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i+1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if (i == 0 and j == 0):#if at the bottom and left edge of lattice
                Temp1 = InitialState[i,j]*InitialState[i,j+1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j+1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i-1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if (i == 0 and j == 0):#if at the bottom and right edge of lattice
                Temp1 = InitialState[i,j]*InitialState[i,j-1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j-1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i-1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if i == 0:#if only at top edge
                Temp1 = InitialState[i,j]*InitialState[i,j+1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j-1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i+1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if i == 0:#if only at bottom edge
                Temp1 = InitialState[i,j]*InitialState[i,j+1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j-1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i-1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if i == 0:#if only at left edge
                Temp1 = InitialState[i,j]*InitialState[i,j+1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j+1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            if i == 0:#if only at right edge
                Temp1 = InitialState[i,j]*InitialState[i,j-1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j-1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
            else:#if somewhere in the middle
                Temp1 = InitialState[i,j]*InitialState[i,j-1]#to the right
                Temp2 = InitialState[i,j]*InitialState[i,j+1]#to the left
                Temp3 = InitialState[i,j]*InitialState[i-1,j]#above
                Temp4 = InitialState[i,j]*InitialState[i+1,j]#below
                Energy += - (Temp1 + Temp2 + Temp3 + Temp4)
                
    return Energy

**The Monte Carlo calculation itself:**

In [46]:
def MonteCarloIsing(InitialState,NumOfCycles,LatticeSize):
    M = LatticeSize
    MC = NumOfCycles
    for i in range(MC):#Looping over the number of MC cycles
        #picking a random coordinate in the lattice
        randNum = np.random.randint(M*M)
        line = int(np.floor(randNum/M))#finding the relavant line in the lattice
        column = int(randNum - M*line)#finding the relavant column in the lattice
        #print(randNum,line,column)
        #flipping the spin at that position
        if InitialState[line,column] == 1:
            InitialState[line,column] = int(-1)
        else:
            InitialState[line,column] = int(1)
        #finding the difference in energy - using periodic boundary conditions
        #NEED to take into account can be on both left/right and top/bottom
        DeltaE = NeigbourInteraction(line,column,InitialState,M)
        
        print(DeltaE)
        

MonteCarloIsing(A,10,4)

-0.0
-2.0
-0.0
4.0
4.0
-4.0
-2.0
-2.0
-0.0
-0.0
